In [1]:
import pandas as pd

### Extract
Extract merupakan proses meng-ekstraksi data dari sumber, sumber data ini bisa berupa relational data (SQL) atau tabel, nonrelational (NoSQL) maupun yang lainnya.

In [8]:
df_participant = pd.read_csv('https://storage.googleapis.com/dqlab-dataset/dqthon-participants.csv')
df_participant.info()
df_participant.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   participant_id  5000 non-null   object 
 1   first_name      5000 non-null   object 
 2   last_name       5000 non-null   object 
 3   birth_date      5000 non-null   object 
 4   address         5000 non-null   object 
 5   phone_number    5000 non-null   object 
 6   country         5000 non-null   object 
 7   institute       5000 non-null   object 
 8   occupation      5000 non-null   object 
 9   register_time   5000 non-null   float64
dtypes: float64(1), object(9)
memory usage: 390.8+ KB


,participant_id,first_name,last_name,birth_date,address,phone_number,country,institute,occupation,register_time
0,bd9b6f88-b84f-4c4d-90f8-b67fe2f1a29a,Citra,Nurdiyanti,05 Feb 1991,"Gg. Monginsidi No. 08\nMedan, Aceh 80734",(0151) 081 2706,Georgia,UD Prakasa Mandasari,Business Intelligence Engineer,1.617634e+09
1,7dfe3391-6f40-47b6-b4db-0c76ebaf5fc3,Aris,Setiawan,11 Jan 1993,"Gg. Rajawali Timur No. 7\nPrabumulih, MA 09434",+62 (036) 461 7027,Korea Utara,Universitas Diponegoro,Frontend Engineer,1.617634e+09
2,19582d7f-b824-4fe5-a517-d5bf573fc768,Cornelia,Handayani,31 Jul 1993,"Jalan Kebonjati No. 0\nAmbon, SS 57739",089 833 6695,Komoro,UD Hardiansyah Puspasari,Business Analyst,1.617634e+09
3,aeb6d817-25f3-4867-8a74-8d92e0a0f633,Soleh,Rajasa,04 Nov 1991,"Jl. Yos Sudarso No. 109\nLubuklinggau, SR 76156",+62 (418) 329-4756,Eritrea,Perum Tampubolon Yuliarti,DevOps Engineer,1.617634e+09
4,1fdabdd9-5444-4c97-87b2-fe8833ad0d27,Vivi,Astuti,22 Jan 2003,"Jalan Gardujati No. 53\nKediri, Sulawesi Tenga...",0812511835,Aljazair,PT Hardiansyah Rahimah,Data Analyst,1.617634e+09


### Transform I : Kode Pos
Membuat kolom baru bernama postal_code yang memuat informasi mengenai kode pos yang diambil dari alamat peserta (kolom address) untuk keperluan pengiriman hadiah agar lebih mudah. Kode pos berada di kalimat paling akhir pada kolom alamat. 

In [6]:
df_participant['postal_code'] = df_participant['address'].str.extract(r'(\d+)$') #regex pada extract berfungsi untuk mengambil angka kode pos di kolom address

### Transform II : Kota
Selain kode pos, dibutuhkan juga kota dari peserta. Untuk menyediakan informasi tersebut, harus membuat kolom baru bernama city yang didapat dari kolom address. Diasumsikan bahwa kota merupakan sekumpulan karakter yang terdapat setelah nomor jalan diikuti dengan \n (newline character) atau lainnya yaitu enter.

In [9]:
df_participant['city'] = df_participant['address'].str.extract(r'(?<=\n)(\w.+)(?=,)')

### Transform III : Github
Salah satu parameter untuk mengetahui proyek apa saja yang pernah dikerjakan oleh peserta yaitu dari git repository mereka dengan menggunakan profil github sebagai parameternya. Sehingga butuh membuat kolom baru bernama github_profile yang merupakan link profil github dari peserta.

Dari dataset, profil github peserta merupakan gabungan dari first_name dan last_name yang sudah di-lowercase. 

In [10]:
df_participant['github_profile'] = 'https://github.com/' + df_participant['first_name'].str.lower() + df_participant['last_name'].str.lower()

### Transform IV : Nomor Handphone
Pada dataset, nomor handphone peserta memiliki format yang berbeda-beda. Maka perlu untuk melakukan cleansing pada data nomor handphone agar memiliki format yang sama

In [12]:
df_participant['cleaned_phone_number'] = df_participant['phone_number'].str.replace(r'^(\+62|62)', '0') #mengubah awalan nomor HP berupa angka 62 atau +62 menjadi  0.
df_participant['cleaned_phone_number'] = df_participant['cleaned_phone_number'].str.replace(r'[()-]', '') #menghapus tanda baca seperti kurung buka, kurung tutup, strip⟶ ()-
df_participant['cleaned_phone_number'] = df_participant['cleaned_phone_number'].str.replace(r'\s+', '') #menghapus spasi pada nomor HP

C:\Users\ThinkPad\AppData\Local\Temp\ipykernel_4604\445933550.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df_participant['cleaned_phone_number'] = df_participant['phone_number'].str.replace(r'^(\+62|62)', '0') #mengubah awalan nomor HP berupa angka 62 atau +62 menjadi  0.
C:\Users\ThinkPad\AppData\Local\Temp\ipykernel_4604\445933550.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_participant['cleaned_phone_number'] = df_participant['cleaned_phone_number'].str.replace(r'[()-]', '') #menghapus tanda baca seperti kurung buka, kurung tutup, strip⟶ ()-
C:\Users\ThinkPad\AppData\Local\Temp\ipykernel_4604\445933550.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df_participant['cleaned_phone_number'] = df_participant['cleaned_phone_number'].str.replace(r'\s+', '') #menghapus spasi pada nomor HP


### Transform V : Nama Tim
Pembuatan nama tim merupakan gabungan nilai dari kolom first_name, last_name, country dan institute.

In [16]:
def func(col):
    abbrev_name = "%s%s"%(col['first_name'][0],col['last_name'][0]) #Singkatan dari Nama Depan dan Nama Belakang dengan mengambil huruf pertama
    country = col['country']
    abbrev_institute = '%s'%(''.join(list(map(lambda word: word[0], col['institute'].split())))) #Singkatan dari value di kolom institute
    return "%s-%s-%s"%(abbrev_name,country,abbrev_institute)

df_participant['team_name'] = df_participant.apply(func, axis=1)

### Transform VI : Email
Dari data peserta yang dimiliki, ada satu informasi yang penting namun belum tersedia, yaitu email. Sehingga perlu menyediakan informasi email dari peserta dengan aturan bahwa format email sebagai berikut:

> 
    Format email:
    xxyy@aa.bb.[ac/com].[cc]

    Keterangan:
    xx -> nama depan (first_name) dalam lowercase
    yy -> nama belakang (last_name) dalam lowercase
    aa -> nama institusi

    Untuk nilai bb, dan cc mengikuti nilai dari aa. Aturannya:
    - Jika institusi nya merupakan Universitas, maka
      bb -> gabungan dari huruf pertama pada setiap kata dari nama Universitas dalam lowercase
      Kemudian, diikuti dengan .ac yang menandakan akademi/institusi belajar dan diikuti dengan pattern cc
    - Jika institusi bukan merupakan Universitas, maka
      bb -> gabungan dari huruf pertama pada setiap kata dari nama Universitas dalam lowercase
      Kemudian, diikuti dengan .com. Perlu diketahui bahwa pattern cc tidak berlaku pada kondisi ini

    cc -> merupakan negara asal peserta, adapun aturannya:
    - Jika banyaknya kata pada negara tersebut lebih dari 1 maka ambil singkatan dari negara tersebut dalam lowercase
    - Namun, jika banyaknya kata hanya 1 maka ambil 3 huruf terdepan dari negara tersebut dalam lowercase

    Contoh:
      Nama depan: Citra
      Nama belakang: Nurdiyanti
      Institusi: UD Prakasa Mandasari
      Negara: Georgia
      Maka,Email nya: citranurdiyanti@upm.geo
      -----------------------------------
      Nama depan: Aris
      Nama belakang: Setiawan
      Institusi: Universitas Diponegoro
      Negara: Korea Utara
      Maka, Email nya: arissetiawan@ud.ac.ku

In [17]:
def func(col):
    first_name_lower = col['first_name'].lower()
    last_name_lower = col['last_name'].lower()
    institute = ''.join(list(map(lambda word: word[0], col['institute'].lower().split()))) #Singkatan dari nama perusahaan dalam lowercase

    if 'Universitas' in col['institute']:
        if len(col['country'].split()) > 1: #Kondisi untuk mengecek apakah jumlah kata dari country lebih dari 1
            country = ''.join(list(map(lambda word: word[0], col['country'].lower().split())))
        else:
            country = col['country'][:3].lower()
        return "%s%s@%s.ac.%s"%(first_name_lower,last_name_lower,institute,country)

    return "%s%s@%s.com"%(first_name_lower,last_name_lower,institute)

df_participant['email'] = df_participant.apply(func, axis=1)

### Transform VII - Tanggal Lahir
Melakukan formatting terhadap kolom birth_date menjadi YYYY-MM-DD dan simpan di kolom yang sama.

In [19]:
df_participant['birth_date'] = pd.to_datetime(df_participant['birth_date'], format='%d %b %Y')

### Transform VII : Tanggal Daftar Kompetisi
merubah register_time ke format DATETIME sesuai dengan aturan dari MySQL. Aturan MySQL: YYYY-MM-DD HH:mm:ss

In [21]:
df_participant['register_at'] = pd.to_datetime(df_participant['register_time'], unit='s')

### Load
Setelah data melalui tahap transform, dataset menjadi telah berubah dari saat tahap extract, ada beberapa kolom tambahan yang dihasilkan dari nilai pada kolom lain.

In [22]:
df_participant.head()

,participant_id,first_name,last_name,birth_date,address,phone_number,country,institute,occupation,register_time,city,github_profile,cleaned_phone_number,team_name,email,register_at
0,bd9b6f88-b84f-4c4d-90f8-b67fe2f1a29a,Citra,Nurdiyanti,1991-02-05,"Gg. Monginsidi No. 08\nMedan, Aceh 80734",(0151) 081 2706,Georgia,UD Prakasa Mandasari,Business Intelligence Engineer,1.617634e+09,Medan,https://github.com/citranurdiyanti,01510812706,CN-Georgia-UPM,citranurdiyanti@upm.com,2021-04-05 14:47:26
1,7dfe3391-6f40-47b6-b4db-0c76ebaf5fc3,Aris,Setiawan,1993-01-11,"Gg. Rajawali Timur No. 7\nPrabumulih, MA 09434",+62 (036) 461 7027,Korea Utara,Universitas Diponegoro,Frontend Engineer,1.617634e+09,Prabumulih,https://github.com/arissetiawan,00364617027,AS-Korea Utara-UD,arissetiawan@ud.ac.ku,2021-04-05 14:46:58
2,19582d7f-b824-4fe5-a517-d5bf573fc768,Cornelia,Handayani,1993-07-31,"Jalan Kebonjati No. 0\nAmbon, SS 57739",089 833 6695,Komoro,UD Hardiansyah Puspasari,Business Analyst,1.617634e+09,Ambon,https://github.com/corneliahandayani,0898336695,CH-Komoro-UHP,corneliahandayani@uhp.com,2021-04-05 14:47:15
3,aeb6d817-25f3-4867-8a74-8d92e0a0f633,Soleh,Rajasa,1991-11-04,"Jl. Yos Sudarso No. 109\nLubuklinggau, SR 76156",+62 (418) 329-4756,Eritrea,Perum Tampubolon Yuliarti,DevOps Engineer,1.617634e+09,Lubuklinggau,https://github.com/solehrajasa,04183294756,SR-Eritrea-PTY,solehrajasa@pty.com,2021-04-05 14:47:14
4,1fdabdd9-5444-4c97-87b2-fe8833ad0d27,Vivi,Astuti,2003-01-22,"Jalan Gardujati No. 53\nKediri, Sulawesi Tenga...",0812511835,Aljazair,PT Hardiansyah Rahimah,Data Analyst,1.617634e+09,Kediri,https://github.com/viviastuti,0812511835,VA-Aljazair-PHR,viviastuti@phr.com,2021-04-05 14:46:50


Pada bagian load ini, data yang sudah ditransformasi sedemikian rupa sehingga sesuai dengan kebutuhan tim analyst dimasukkan kembali ke database yaitu Data Warehouse (DWH). Biasanya, dilakukan pendefinisian skema database terlebih dahulu seperti:

Nama kolom
Tipe kolom
Apakah primary key, unique key, index atau bukan
Panjang kolomnya
Karena umumnya Data Warehouse merupakan database yang terstruktur sehingga mereka memerlukan skema sebelum datanya dimasukkan.

Pandas sudah menyediakan fungsi untuk memasukkan data ke database yaitu to_sql().

Detail dari fungsi tersebut bisa dilihat pada dokumentasi Pandas: https://pandas.pydata.org/docs/ref